# Online Evaluations: Evaluating in the Cloud on a Schedule

## Objective

This tutorial provides a step-by-step guide on how to evaluate data generated by LLMs online on a schedule. 

This tutorial uses the following Azure AI services:

- [Azure AI Safety Evaluation](https://aka.ms/azureaistudiosafetyeval)
- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates the online evaluation of a LLM. It is important to have access to AzureOpenAI credentials and an AzureAI project. This example demonstrates: 

- Recurring, Online Evaluation (to be used to monitor LLMs once they are deployed)

## Before you begin
### Prerequesite
- Configure resources to support Online Evaluation as per [Online Evaluation documentation](https://aka.ms/GenAIMonitoringDoc)

In [ ]:
%pip install -U azure-identity
%pip install -U azure-ai-projects
%pip install -U azure-ai-evaluation

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import (
    ApplicationInsightsConfiguration,
    EvaluatorConfiguration,
    ConnectionType,
    EvaluationSchedule,
    RecurrenceTrigger,
)
from azure.ai.evaluation import F1ScoreEvaluator, ViolenceEvaluator

### Connect to your Azure Open AI deployment
To evaluate your LLM-generated data remotely in the cloud, we must connect to your Azure Open AI deployment. This deployment must be a GPT model which supports `chat completion`, such as `gpt-4`. To see the proper value for `conn_str`, navigate to the connection string at the "Project Overview" page for your Azure AI project. 

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str="<connection_string>",  # At the moment, it should be in the format "<Region>.api.azureml.ms;<AzureSubscriptionId>;<ResourceGroup>;<HubName>" Ex: eastus2.api.azureml.ms;xxxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxxxxx;rg-sample;sample-project-eastus2
)

Please see [Online Evaluation documentation](https://aka.ms/GenAIMonitoringDoc) for configuration of Application Insights. `service_name` is a unique name you provide to define your Generative AI application and identify it within your Application Insights resource. This property will be logged in the `traces` table in Application Insights and can be found in the `customDimensions["service.name"]` field. `evaluation_name` is a unique name you provide for your Online Evaluation schedule. 

In [ ]:
# Your Application Insights resource ID
# At the moment, it should be something in the format "/subscriptions/<AzureSubscriptionId>/resourceGroups/<ResourceGroup>/providers/Microsoft.Insights/components/<ApplicationInsights>""
app_insights_resource_id = "<app_insights_resource_id>"

# Name of your generative AI application (will be available in trace data in Application Insights)
service_name = "<service_name>"

# Name of your online evaluation schedule
evaluation_name = "<evaluation_name>"

Below is the Kusto Query Language (KQL) query to query data from Application Insights resource. This query is compatible with data logged by the Azure AI Inferencing Tracing SDK (linked in [documentation](https://aka.ms/GenAIMonitoringDoc)). You can modify it depending on your data schema. The KQL query must output several columns: `operation_ID`, `operation_ParentID`, and `gen_ai_response_id`. You can choose which other columns to output as required by the evaluators you are using.

In [ ]:
kusto_query = 'let gen_ai_spans=(dependencies | where isnotnull(customDimensions["gen_ai.system"]) | extend response_id = tostring(customDimensions["gen_ai.response.id"]) | project id, operation_Id, operation_ParentId, timestamp, response_id); let gen_ai_events=(traces | where message in ("gen_ai.choice", "gen_ai.user.message", "gen_ai.system.message") or tostring(customDimensions["event.name"]) in ("gen_ai.choice", "gen_ai.user.message", "gen_ai.system.message") | project id= operation_ParentId, operation_Id, operation_ParentId, user_input = iff(message == "gen_ai.user.message" or tostring(customDimensions["event.name"]) == "gen_ai.user.message", parse_json(iff(message == "gen_ai.user.message", tostring(customDimensions["gen_ai.event.content"]), message)).content, ""), system = iff(message == "gen_ai.system.message" or tostring(customDimensions["event.name"]) == "gen_ai.system.message", parse_json(iff(message == "gen_ai.system.message", tostring(customDimensions["gen_ai.event.content"]), message)).content, ""), llm_response = iff(message == "gen_ai.choice", parse_json(tostring(parse_json(tostring(customDimensions["gen_ai.event.content"])).message)).content, iff(tostring(customDimensions["event.name"]) == "gen_ai.choice", parse_json(parse_json(message).message).content, "")) | summarize operation_ParentId = any(operation_ParentId), Input = maxif(user_input, user_input != ""), System = maxif(system, system != ""), Output = maxif(llm_response, llm_response != "") by operation_Id, id); gen_ai_spans | join kind=inner (gen_ai_events) on id, operation_Id | project Input, System, Output, operation_Id, operation_ParentId, gen_ai_response_id = response_id'

# AzureMSIClientId is the clientID of the User-assigned managed identity created during set-up - see documentation for how to find it
properties = {"AzureMSIClientId": "your_client_id"}

In [ ]:
# Connect to your Application Insights resource
app_insights_config = ApplicationInsightsConfiguration(
    resource_id=app_insights_resource_id, query=kusto_query, service_name=service_name
)

In [ ]:
# Connect to your AOAI resource, you must use an AOAI GPT model
deployment_name = "gpt-4"
api_version = "2024-06-01"
default_connection = project_client.connections.get_default(connection_type=ConnectionType.AZURE_OPEN_AI)
model_config = default_connection.to_evaluator_model_config(deployment_name=deployment_name, api_version=api_version)

### Configure Evaluators to Run
The code below demonstrates how to configure the evaluators you want to run. In this example, we use the `F1ScoreEvaluator`, `RelevanceEvaluator` and the `ViolenceEvaluator`, but all evaluators supported by [Azure AI Evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/evaluation-metrics-built-in?tabs=warning) are supported by Online Evaluation and can be configured here. You can either import the classes from the SDK and reference them with the `.id` property, or you can find the fully formed `id` of the evaluator in the AI Studio registry of evaluators, and use it here. 

In [ ]:
# id for each evaluator can be found in your AI Studio registry - please see documentation for more information
# init_params is the configuration for the model to use to perform the evaluation
# data_mapping is used to map the output columns of your query to the names required by the evaluator
evaluators = {
    "f1_score": EvaluatorConfiguration(
        id=F1ScoreEvaluator.id,
    ),
    "relevance": EvaluatorConfiguration(
        id="azureml://registries/azureml-staging/models/Relevance-Evaluator/versions/4",
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.Input}", "response": "${data.Output}"},
    ),
    "violence": EvaluatorConfiguration(
        id=ViolenceEvaluator.id,
        init_params={"azure_ai_project": project_client.scope},
        data_mapping={"query": "${data.Input}", "response": "${data.Output}"},
    ),
}

### Evaluate in the Cloud on a Schedule with Online Evaluation

You can configure the `RecurrenceTrigger` based on the class definition [here](https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.recurrencetrigger?view=azure-python).

In [ ]:
# Frequency to run the schedule
recurrence_trigger = RecurrenceTrigger(frequency="day", interval=1)

# Configure the online evaluation schedule
evaluation_schedule = EvaluationSchedule(
    data=app_insights_config,
    evaluators=evaluators,
    trigger=recurrence_trigger,
    description=f"{service_name} evaluation schedule",
    properties=properties,
)

# Create the online evaluation schedule
created_evaluation_schedule = project_client.evaluations.create_or_replace_schedule(service_name, evaluation_schedule)
print(
    f"Successfully submitted the online evaluation schedule creation request - {created_evaluation_schedule.name}, currently in {created_evaluation_schedule.provisioning_state} state."
)

### Next steps 

Navigate to the "Tracing" tab in [Azure AI Studio](https://ai.azure.com/) to view your logged trace data alongside the evaluations produced by the Online Evaluation schedule. You can use the reference link provided in the "Tracing" tab to navigate to a comprehensive workbook in Application Insights for more details on how your application is performing. 